In [2]:
import io
from pydub import AudioSegment
import asyncio
import websockets
import json
import base64
import os
from dotenv import load_dotenv
from IPython.display import Audio, display
from websockets.exceptions import ConnectionClosedError

# Load API key from .env file
load_dotenv()
API_KEY = os.getenv("HUME_AI_API_KEY")
API_URL = "wss://api.hume.ai/v0/evi/chat"

print(f"API Key: {'*' * len(API_KEY)}")  # Print masked API key for verification

API Key: ************************************************


In [3]:
async def send_message(websocket, message):
    assistant_input = {
        "type": "assistant_input",
        "text": message
    }
    await websocket.send(json.dumps(assistant_input))
    print(f"Message sent: {message}")

async def receive_audio(websocket):
    print("Waiting for audio response...")
    audio_chunks = []
    text_response = ""
    
    try:
        while True:
            response = await websocket.recv()
            data = json.loads(response)
            print(f"Received response of type: {data['type']}")
            
            if data["type"] == "audio_output":
                audio_chunks.append(base64.b64decode(data["data"]))
            elif data["type"] == "assistant_message":
                text_response += data['message']['content'] + " "
            elif data["type"] == "assistant_end":
                print("Received end of assistant response")
                break
            else:
                print(f"Received other message type: {data['type']}")
    
    except ConnectionClosedError:
        print("Connection closed unexpectedly. The complete message may not have been received.")
    
    if audio_chunks:
        # Combine all audio chunks
        combined_audio = AudioSegment.empty()
        for chunk in audio_chunks:
            segment = AudioSegment.from_wav(io.BytesIO(chunk))
            combined_audio += segment
        
        # Save complete audio to a temporary file
        temp_file = f"temp_audio_combined.wav"
        combined_audio.export(temp_file, format="wav")
        
        print(f"Complete audio saved as {temp_file}")
        
        # Play the complete audio
        display(Audio(temp_file, autoplay=True))
        
        # Clean up the temporary file
        os.remove(temp_file)
    else:
        print("No audio data received")
    
    print("Assistant's text response:")
    print(text_response.strip())

async def connect_to_evi(max_retries=3, retry_delay=5):
    uri = f"{API_URL}?api_key={API_KEY}"
    for attempt in range(max_retries):
        try:
            websocket = await websockets.connect(uri)
            print("Connected to Hume AI EVI Chat API")
            return websocket
        except Exception as e:
            print(f"Connection attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                await asyncio.sleep(retry_delay)
            else:
                print("Max retries reached. Unable to connect.")
    return None

async def chat_once(websocket, message):
    if websocket:
        try:
            await send_message(websocket, message)
            await receive_audio(websocket)
        except ConnectionClosedError:
            print("Connection closed unexpectedly. Trying to reconnect...")
            websocket = await connect_to_evi()
            if websocket:
                await chat_once(websocket, message)
            else:
                print("Failed to reconnect. Please try again later.")
    else:
        print("No active connection. Please reconnect.")

async def close_connection(websocket):
    if websocket:
        try:
            await websocket.close()
            print("Connection closed")
        except Exception as e:
            print(f"Error closing connection: {e}")

In [10]:
# Connect to the API
websocket = await connect_to_evi()

Connected to Hume AI EVI Chat API


In [5]:
# Send a message and receive a response
user_input = input("You: ")
await chat_once(websocket, user_input)

Message sent: hellloooooo... how are you Mumbai?
Waiting for audio response...
Received response of type: chat_metadata
Received other message type: chat_metadata
Received response of type: assistant_message
Received response of type: audio_output
Received response of type: assistant_message
Received response of type: audio_output
Received response of type: assistant_end
Received end of assistant response
Complete audio saved as temp_audio_combined.wav


Assistant's text response:
hellloooooo... how are you Mumbai?


In [6]:
# Close the connection when you're done
await close_connection(websocket)

Connection closed


In [7]:
msg = "I'm so excited to finally achieve my lifelong dream of becoming a pilot! After years of hard work and countless setbacks, I passed my final test today. I can't stop smiling!"